# WaveletDiff Evaluation (Refactored)

This notebook evaluates a trained WaveletDiff model. It acts as a frontend interface, delegating heavy logic to `src/evaluation` modules.

In [ ]:
# @title Cell 1: Global Configuration
import os

# --- Drive Paths ---
DRIVE_MOUNT_PATH = "/content/drive" # @param {type:"string"}
DRIVE_BASE_PATH = "/content/drive/MyDrive/personal_drive/trading/waveletDiff" # @param {type:"string"}
CHECKPOINT_FOLDER = "checkpoints" # @param {type:"string"}
SAMPLES_FOLDER = "samples" # @param {type:"string"}

MODEL_FILENAME = "stocks_experiment.tar.gz" # @param {type:"string"}
MODEL_BASENAME = MODEL_FILENAME.replace('.tar.gz', '').replace('.zip', '').replace('.ckpt', '').replace('.tgz', '').replace('.gz', '')

DRIVE_CHECKPOINT_PATH = os.path.join(DRIVE_BASE_PATH, CHECKPOINT_FOLDER, MODEL_FILENAME)
DRIVE_SAMPLES_PATH = os.path.join(DRIVE_BASE_PATH, SAMPLES_FOLDER, MODEL_BASENAME)

# --- Repository Settings ---
REPO_BRANCH = "develop" # @param {type:"string"}

# --- Evaluation Settings ---
DATASET = "stocks" # @param {type:"string"}
EXPERIMENT_NAME = "evaluation_run" # @param {type:"string"}
NUM_SAMPLES = 2000 # @param {type:"integer"}
SAMPLING_METHOD = "ddpm" # @param ["ddpm", "ddim"]
COMPILE_MODE = "none" # @param ["none", "default", "reduce-overhead", "max-autotune"]
DEVICE = "cuda" # @param ["cuda", "cpu"]

# --- Evaluation Options ---
EXCLUDE_VOLUME = True # @param {type:"boolean"}
CACHE_SAMPLES_TO_DRIVE = True # @param {type:"boolean"}
USE_CACHED_SAMPLES = True # @param {type:"boolean"}

In [ ]:
# @title Cell 2: Setup (Clone, Install, Mount)
import os
import sys
import shutil
from google.colab import drive

REPO_URL = "https://github.com/MilesHoffman/waveletDiff_synth_data.git"
REPO_NAME = "waveletDiff_synth_data"
REPO_PATH = os.path.abspath(REPO_NAME)

# 1. Clone or Pull Repo
if os.path.exists(REPO_PATH):
    nested_path = os.path.join(REPO_PATH, REPO_NAME)
    if os.path.exists(nested_path):
        shutil.rmtree(REPO_PATH)
        !git clone {REPO_URL} {REPO_NAME}
        !git -C {REPO_NAME} checkout {REPO_BRANCH}
    else:
        !git -C {REPO_NAME} fetch origin
        !git -C {REPO_NAME} checkout {REPO_BRANCH}
        !git -C {REPO_NAME} pull origin {REPO_BRANCH}
else:
    !git clone {REPO_URL} {REPO_NAME}
    !git -C {REPO_NAME} checkout {REPO_BRANCH}

# 2. Install Dependencies
!pip install -q pytorch-lightning pywavelets scipy pandas tqdm scikit-learn tslearn seaborn statsmodels

# 3. Mount Drive & Setup Paths
if not os.path.exists(DRIVE_MOUNT_PATH):
    drive.mount(DRIVE_MOUNT_PATH)

for p in [os.path.join(REPO_PATH, "src"), os.path.join(REPO_PATH, "src", "evaluation")]:
    if p not in sys.path: sys.path.append(p)

# 4. Prepare Experiments
local_exp_dir = os.path.join(REPO_PATH, "outputs", EXPERIMENT_NAME)
os.makedirs(local_exp_dir, exist_ok=True)

if os.path.exists(DRIVE_CHECKPOINT_PATH):
    print(f"Unpacking model from {DRIVE_CHECKPOINT_PATH}...")
    if DRIVE_CHECKPOINT_PATH.endswith(".ckpt"):
        shutil.copy2(DRIVE_CHECKPOINT_PATH, os.path.join(local_exp_dir, "checkpoint.ckpt"))
    else:
        shutil.unpack_archive(DRIVE_CHECKPOINT_PATH, local_exp_dir, format='gztar' if '.gz' in DRIVE_CHECKPOINT_PATH and not '.tar' in DRIVE_CHECKPOINT_PATH else None)
else:
    print(f"❌ Model file not found.")

# Sync configs
if os.path.exists(os.path.join(REPO_PATH, "WaveletDiff_source", "configs")):
    shutil.rmtree(os.path.join(REPO_PATH, "configs"), ignore_errors=True)
    shutil.copytree(os.path.join(REPO_PATH, "WaveletDiff_source", "configs"), os.path.join(REPO_PATH, "configs"))

print("✅ Setup Complete")

In [ ]:
# @title Cell 3: Generate or Load Samples
import numpy as np
os.chdir(os.path.join(REPO_PATH, "src"))

# Dollar space paths
local_gen_path = f"../outputs/{EXPERIMENT_NAME}/{SAMPLING_METHOD}_samples.npy"
local_real_path = f"../outputs/{EXPERIMENT_NAME}/real_samples.npy"
drive_gen_path = os.path.join(DRIVE_SAMPLES_PATH, f"{SAMPLING_METHOD}_samples.npy")
drive_real_path = os.path.join(DRIVE_SAMPLES_PATH, "real_samples.npy")

# Reparameterized (norm) space paths
local_gen_norm_path = f"../outputs/{EXPERIMENT_NAME}/{SAMPLING_METHOD}_samples_norm.npy"
local_real_norm_path = f"../outputs/{EXPERIMENT_NAME}/real_samples_norm.npy"
drive_gen_norm_path = os.path.join(DRIVE_SAMPLES_PATH, f"{SAMPLING_METHOD}_samples_norm.npy")
drive_real_norm_path = os.path.join(DRIVE_SAMPLES_PATH, "real_samples_norm.npy")

if USE_CACHED_SAMPLES and os.path.exists(drive_gen_path):
    print("Loading cached samples...")
    os.makedirs(os.path.dirname(local_gen_path), exist_ok=True)
    shutil.copy2(drive_gen_path, local_gen_path)
    shutil.copy2(drive_real_path, local_real_path)
    # Copy norm files if they exist
    if os.path.exists(drive_gen_norm_path):
        shutil.copy2(drive_gen_norm_path, local_gen_norm_path)
        shutil.copy2(drive_real_norm_path, local_real_norm_path)
else:
    print(f"Generating {NUM_SAMPLES} samples...")
    output = !python sample.py --experiment_name {EXPERIMENT_NAME} --dataset {DATASET} --num_samples {NUM_SAMPLES} --sampling_method {SAMPLING_METHOD} --compile_mode {COMPILE_MODE}
    print("\n".join(output))
    
    if not os.path.exists(local_gen_path):
        print("\n❌ Generation failed: Output file not produced.")
        print("Possible causes: Checkpoint not found, CUDA error, or config mismatch.")
        raise FileNotFoundError(f"File does not exist: {local_gen_path}")

    if CACHE_SAMPLES_TO_DRIVE:
        os.makedirs(DRIVE_SAMPLES_PATH, exist_ok=True)
        shutil.copy2(local_gen_path, drive_gen_path)
        shutil.copy2(local_real_path, drive_real_path)
        # Cache norm files if they exist
        if os.path.exists(local_gen_norm_path):
            shutil.copy2(local_gen_norm_path, drive_gen_norm_path)
            shutil.copy2(local_real_norm_path, drive_real_norm_path)

print("✅ Samples Ready")

In [ ]:
# @title Cell 4: Initialize Modules
import sys
import numpy as np
import torch
import matplotlib.pyplot as plt

# New Refactored Modules
from evaluation import visualizations as viz
from evaluation import statistics as stats
from evaluation import reporting as report
from evaluation import wrappers
from training import inline_evaluation

# Load Dollar Space Data
real_path = f"../outputs/{EXPERIMENT_NAME}/real_samples.npy"
gen_path = f"../outputs/{EXPERIMENT_NAME}/{SAMPLING_METHOD}_samples.npy"
real_data_full = np.load(real_path)
generated_data_full = np.load(gen_path)

print(f"Loaded Full OHLCV (Dollar): Real {real_data_full.shape}, Gen {generated_data_full.shape}")

# Load Reparameterized (Norm) Space Data
real_norm_path = f"../outputs/{EXPERIMENT_NAME}/real_samples_norm.npy"
gen_norm_path = f"../outputs/{EXPERIMENT_NAME}/{SAMPLING_METHOD}_samples_norm.npy"
HAS_NORM_DATA = os.path.exists(real_norm_path) and os.path.exists(gen_norm_path)

if HAS_NORM_DATA:
    real_data_norm_full = np.load(real_norm_path)
    generated_data_norm_full = np.load(gen_norm_path)
    print(f"Loaded Reparameterized: Real {real_data_norm_full.shape}, Gen {generated_data_norm_full.shape}")
else:
    print("⚠️ Reparameterized data not found. Re-run sample.py with updated code to generate.")
    real_data_norm_full = None
    generated_data_norm_full = None

# Downsample for metrics (keep indices consistent)
n_s = min(2000, len(real_data_full), len(generated_data_full))
np.random.seed(42)
real_idx = np.random.choice(len(real_data_full), n_s, replace=False)
gen_idx = np.random.choice(len(generated_data_full), n_s, replace=False)

# Full OHLCV for sample visualizations (Dollar Space)
real_data_ohlcv = real_data_full[real_idx]
generated_data_ohlcv = generated_data_full[gen_idx]

# Reparameterized subsets
if HAS_NORM_DATA:
    real_data_norm = real_data_norm_full[real_idx]
    generated_data_norm = generated_data_norm_full[gen_idx]
    
    # Scale norm data for discriminative/predictive benchmarks
    norm_min = np.min(real_data_norm, axis=(0,1), keepdims=True)
    norm_max = np.max(real_data_norm, axis=(0,1), keepdims=True)
    real_data_norm_scaled = (real_data_norm - norm_min) / (norm_max - norm_min + 1e-8)
    generated_data_norm_scaled = (generated_data_norm - norm_min) / (norm_max - norm_min + 1e-8)

# Apply EXCLUDE_VOLUME for metrics only
if EXCLUDE_VOLUME and real_data_full.shape[2] > 1:
    real_data = real_data_ohlcv[..., :-1]
    generated_data = generated_data_ohlcv[..., :-1]
else:
    real_data = real_data_ohlcv
    generated_data = generated_data_ohlcv

# Prepare Raw vs Scaled (for metrics)
real_data_raw, generated_data_raw = real_data.copy(), generated_data.copy()

dmin, dmax = np.min(real_data, axis=(0,1), keepdims=True), np.max(real_data, axis=(0,1), keepdims=True)
real_data_scaled = (real_data - dmin) / (dmax - dmin + 1e-8)
generated_data_scaled = (generated_data - dmin) / (dmax - dmin + 1e-8)

print(f"Metrics Data (EXCLUDE_VOLUME={EXCLUDE_VOLUME}): {real_data.shape}")

### Visualizations

In [ ]:
# @title Visual Analysis
# t-SNE and PCA use scaled OHLC data (no volume)
viz.plot_distribution_reduction(real_data_scaled, generated_data_scaled)
viz.plot_pdf(real_data_scaled, generated_data_scaled)

# Sample plots use FULL OHLCV data in Dollar Space
viz.plot_candlesticks(real_data_ohlcv, generated_data_ohlcv)
viz.plot_samples(real_data_ohlcv, generated_data_ohlcv)

### Comprehensive Metrics Evaluation

Evaluates model performance across both Dollar Space and Reparameterized (Norm) Space.
Includes Fidelity (Distribution), Utility (Predictive), and Diversity (Coverage/Memorization) metrics.

In [ ]:
# @title Run Evaluation Metrics (Dual Space)
from scipy.stats import wasserstein_distance
import warnings
warnings.filterwarnings('ignore')

metrics_dict = {}

# ==========================================
# 1. DOLLAR SPACE EVALUATION
# ==========================================
if generated_data is not None:
    print("\n" + "="*50)
    print("EVALUATION: DOLLAR SPACE (Raw Prices)")
    print("="*50)
    
    # A. Statistical
    stat_agg, real_stat_det, gen_stat_det = stats.calculate_statistical_metrics(real_data_raw, generated_data_raw)
    metrics_dict['stat_results'] = stat_agg
    
    # B. Utility (TSTR vs TRTR)
    print("\nRunning Utility Checks (TSTR vs TRTR)...")
    # TSTR: Train on Synthetic, Test on Real
    tstr_score = wrappers.run_predictive_benchmark(real_data, generated_data, iterations=1)[0]
    # TRTR: Train on Real, Test on Real (Baseline)
    trtr_score = wrappers.run_predictive_benchmark(real_data, real_data, iterations=1)[0]
    
    metrics_dict['predictive_score'] = [tstr_score]
    # metrics_dict['utility_trtr'] = trtr_score
    print(f"  TSTR (Score): {tstr_score:.6f}")
    print(f"  TRTR (Base):  {trtr_score:.6f}")
    print(f"  Utility Gap:  {abs(tstr_score - trtr_score):.6f} (Lower is better)")

    # C. Discriminative Score
    disc_scores = wrappers.run_discriminative_benchmark(real_data, generated_data, iterations=1)
    metrics_dict['discriminative_score'] = disc_scores
    
    # D. Advanced Metrics (Fidelity, Diversity)
    dist_res, struct_res, fin_res = wrappers.run_advanced_metrics(real_data, generated_data)
    mem_ratio, div_res, fld_score, dcr, prec, recall, mmd = wrappers.run_new_metrics(real_data, generated_data)
    
    # Store results
    metrics_dict['dist_results'] = dist_res
    metrics_dict['struct_results'] = struct_res
    metrics_dict['fin_results'] = fin_res
    metrics_dict['mem_ratio'] = mem_ratio
    metrics_dict['div_results'] = div_res
    metrics_dict['fld_score'] = fld_score
    metrics_dict['dcr_score'] = dcr
    metrics_dict['precision'] = prec
    metrics_dict['recall'] = recall
    metrics_dict['mmd_score'] = mmd
    
    # Display Scorecard (Dollar)
    print("\n--- DOLLAR SPACE SCORECARD ---")
    dollar_scorecard = report.generate_summary_scorecard(metrics_dict)
    display(dollar_scorecard)

# ==========================================
# 2. REPARAMETERIZED SPACE EVALUATION
# ==========================================
if HAS_NORM_DATA:
    print("\n" + "="*50)
    print("EVALUATION: REPARAMETERIZED SPACE (Norm Features)")
    print("="*50)
    
    norm_metrics_dict = {}
    
    # 1. OHLC Invariants (Wick Non-negativity)
    # Assuming index 2=HighNorm, 3=LowNorm based on standard reparam
    if generated_data_norm.shape[2] >= 4:
        wick_high = generated_data_norm[:, :, 2]
        wick_low = generated_data_norm[:, :, 3]
        ohlc_valid_pct = ((wick_high >= -1e-5) & (wick_low >= -1e-5)).mean() * 100
        print(f"OHLC Invariants: {ohlc_valid_pct:.1f}% valid (Wicks >= 0)")
    
    # 2. Run Full Advanced Suite on Norm Data
    # Use raw norm data for distribution checks
    dist_res_n, struct_res_n, fin_res_n = wrappers.run_advanced_metrics(real_data_norm, generated_data_norm)
    mem_ratio_n, div_res_n, fld_score_n, dcr_n, prec_n, recall_n, mmd_n = wrappers.run_new_metrics(real_data_norm, generated_data_norm)
    
    # Store in separate dict
    norm_metrics_dict['dist_results'] = dist_res_n
    norm_metrics_dict['struct_results'] = struct_res_n
    norm_metrics_dict['fin_results'] = fin_res_n
    norm_metrics_dict['mem_ratio'] = mem_ratio_n
    norm_metrics_dict['div_results'] = div_res_n
    norm_metrics_dict['fld_score'] = fld_score_n
    norm_metrics_dict['dcr_score'] = dcr_n
    norm_metrics_dict['precision'] = prec_n
    norm_metrics_dict['recall'] = recall_n
    norm_metrics_dict['mmd_score'] = mmd_n
    
    # Discriminative on Reparam (Texture check)
    # Use scaled norm data if needed, but norm data is usually already small scale. 
    # Let's use real_data_norm directly as it's the raw output.
    disc_score_n = wrappers.run_discriminative_benchmark(real_data_norm, generated_data_norm, iterations=1)
    norm_metrics_dict['discriminative_score'] = disc_score_n
    
    # Display Scorecard (Reparam)
    print("\n--- REPARAMETERIZED SPACE SCORECARD ---")
    norm_scorecard = report.generate_summary_scorecard(norm_metrics_dict)
    display(norm_scorecard)
else:
    print("⚠️ Reparameterized data not available. Skipping norm space metrics.")

In [ ]:
# @title Detailed Feature Stats (Dollar Space)
report.display_feature_stats(real_stat_det, gen_stat_det)